# Analysis OPS

## init

In [12]:
from analysis.ops_latency import OpsAnalyzer

df_list = OpsAnalyzer(counts=500, size=(256, 16, 224, 224)).process()

2020-04-29 05:17:28:pid_373:ops_latency:<INFO> ------------------------------MyModel1------------------------------
2020-04-29 05:30:20:pid_373:ops_latency:<INFO> [torch.Size([256, 16, 224, 224]), torch.Size([256, 16, 224, 224]), torch.Size([256, 16, 224, 224]), torch.Size([256, 16, 224, 224])]
2020-04-29 05:30:20:pid_373:ops_latency:<INFO> ------------------------------MyModel3------------------------------
2020-04-29 05:43:30:pid_373:ops_latency:<INFO> [torch.Size([256, 16, 224, 224]), torch.Size([256, 16, 224, 224]), torch.Size([256, 16, 224, 224]), torch.Size([256, 16, 224, 224])]


### origin

In [13]:
df_list[0]

,5x5 conv_time,3x3 conv_time,7x7 conv_time,total_time
0,538233.041763,329560.756683,720063.447952,1.587857e+06
1,467195.034027,300046.682358,721051.216125,1.488293e+06
2,467411.279678,299165.725708,720895.767212,1.487473e+06
3,467185.497284,299643.039703,721242.189407,1.488071e+06
4,466884.374619,299467.325211,720308.780670,1.486660e+06
...,...,...,...,...
495,469584.226608,300527.572632,721929.073334,1.492041e+06
496,469696.283340,299990.653992,721322.059631,1.491009e+06
497,469706.296921,300850.629807,721863.746643,1.492421e+06
498,468706.130981,300281.524658,721267.700195,1.490255e+06


In [14]:
df_list[0].describe()

,5x5 conv_time,3x3 conv_time,7x7 conv_time,total_time
count,500.000000,500.000000,500.000000,5.000000e+02
mean,469330.148220,300422.009945,721677.831650,1.491430e+06
std,4477.746521,1365.245755,1296.660042,5.698055e+03
min,466884.374619,299083.948135,720003.128052,1.486660e+06
25%,468673.944473,300107.359886,721075.415611,1.490072e+06
50%,468956.708908,300333.619118,721461.296082,1.490680e+06
75%,469208.300114,300593.316555,721890.747547,1.491753e+06
max,538233.041763,329560.756683,738509.416580,1.587857e+06


In [15]:
df_list[1].describe()

,7x7 conv_time,3x3 conv_time,5x5 conv_time,total_time
count,500.000000,500.000000,500.000000,5.000000e+02
mean,739822.276115,307836.266041,480804.108143,1.528463e+06
std,48279.962672,10831.784946,22230.468697,7.970086e+04
min,716417.312622,299297.809601,468296.766281,1.485136e+06
25%,717806.935310,300480.782986,469097.435474,1.487325e+06
50%,724791.169167,306818.366051,475209.712982,1.507701e+06
75%,725926.995277,307756.125927,476067.721844,1.509596e+06
max,917899.847031,353589.057922,566695.451736,1.826865e+06


### remove values more than 90%

In [16]:
# from util.outlier import cut_outlier
import functools

def cut_outlier(x, min_border, max_border):
    print(x.quantile([min_border, max_border]))
    return x[x.apply(lambda y: (y.quantile(min_border) < y) & (y < y.quantile(max_border)), axis=0)]

cut_list = list(map(functools.partial(cut_outlier, min_border=0.25, max_border=0.75), df_list))

      5x5 conv_time  3x3 conv_time  7x7 conv_time    total_time
0.25  468673.944473  300107.359886  721075.415611  1.490072e+06
0.75  469208.300114  300593.316555  721890.747547  1.491753e+06
      7x7 conv_time  3x3 conv_time  5x5 conv_time    total_time
0.25  717806.935310  300480.782986  469097.435474  1.487325e+06
0.75  725926.995277  307756.125927  476067.721844  1.509596e+06


In [17]:
cut_list[0]

,5x5 conv_time,3x3 conv_time,7x7 conv_time,total_time
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,721242.189407,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
495,NaN,300527.572632,NaN,NaN
496,NaN,NaN,721322.059631,1.491009e+06
497,NaN,NaN,721863.746643,NaN
498,468706.130981,300281.524658,721267.700195,1.490255e+06


In [18]:
cut_list[0].describe()

,5x5 conv_time,3x3 conv_time,7x7 conv_time,total_time
count,250.000000,250.000000,250.000000,2.490000e+02
mean,468951.894760,300338.548660,721473.138809,1.490755e+06
std,150.200123,130.597169,227.877899,4.478014e+02
min,468675.851822,300107.479095,721075.773239,1.490072e+06
25%,468835.651875,300232.112408,721279.561520,1.490374e+06
50%,468956.708908,300333.619118,721461.296082,1.490666e+06
75%,469068.408012,300442.993641,721685.707569,1.491118e+06
max,469208.240509,300592.422485,721889.495850,1.491735e+06


In [19]:
cut_list[0].describe().T[0:3]["mean"].sum()

1490763.5822296143

In [20]:
cut_list[1].describe()

,7x7 conv_time,3x3 conv_time,5x5 conv_time,total_time
count,250.000000,250.000000,250.000000,2.500000e+02
mean,722077.791214,304243.702888,472775.808334,1.499197e+06
std,3554.059784,3296.681708,3112.564998,1.008864e+04
min,717807.292938,300482.273102,469099.998474,1.487327e+06
25%,718117.594719,300760.507584,469366.490841,1.488216e+06
50%,724791.169167,306818.366051,475209.712982,1.507701e+06
75%,725414.872169,307423.174381,475734.293461,1.508687e+06
max,725924.491882,307754.516602,476063.489914,1.509595e+06


In [21]:
cut_list[1].describe().T[0:3]["mean"].sum()

1499097.3024368286

## batch_size: 1

In [5]:
from analysis.ops_latency import OpsAnalyzer

df_list1 = OpsAnalyzer(counts=500, size=(1, 16, 224, 224)).process()

2020-05-01 14:12:14:pid_7245:ops_latency:<INFO> ------------------------------MyModel1------------------------------
2020-05-01 14:12:17:pid_7245:ops_latency:<INFO> [torch.Size([1, 16, 224, 224]), torch.Size([1, 16, 224, 224]), torch.Size([1, 16, 224, 224]), torch.Size([1, 16, 224, 224])]
2020-05-01 14:12:17:pid_7245:ops_latency:<INFO> ------------------------------MyModel3------------------------------
2020-05-01 14:12:20:pid_7245:ops_latency:<INFO> [torch.Size([1, 16, 224, 224]), torch.Size([1, 16, 224, 224]), torch.Size([1, 16, 224, 224]), torch.Size([1, 16, 224, 224])]


In [6]:
df_list1[0].describe()

,5x5 conv_time,3x3 conv_time,7x7 conv_time,total_time
count,500.000000,500.000000,500.000000,500.000000
mean,2034.279823,1061.786175,2801.761150,5897.827148
std,154.642961,39.918591,190.000876,332.685645
min,1810.789108,1022.577286,2732.992172,5651.235580
25%,1881.599426,1043.260098,2761.363983,5724.668503
50%,2098.083496,1055.479050,2771.019936,5912.542343
75%,2122.402191,1067.876816,2782.404423,5943.834782
max,3091.812134,1425.743103,4699.468613,9120.941162


In [7]:
df_list32 = OpsAnalyzer(counts=500, size=(32, 16, 224, 224)).process()

2020-05-01 14:12:29:pid_7245:ops_latency:<INFO> ------------------------------MyModel1------------------------------
2020-05-01 14:14:09:pid_7245:ops_latency:<INFO> [torch.Size([32, 16, 224, 224]), torch.Size([32, 16, 224, 224]), torch.Size([32, 16, 224, 224]), torch.Size([32, 16, 224, 224])]
2020-05-01 14:14:09:pid_7245:ops_latency:<INFO> ------------------------------MyModel3------------------------------
2020-05-01 14:15:48:pid_7245:ops_latency:<INFO> [torch.Size([32, 16, 224, 224]), torch.Size([32, 16, 224, 224]), torch.Size([32, 16, 224, 224]), torch.Size([32, 16, 224, 224])]


In [9]:
df_list32[0].describe()

,5x5 conv_time,3x3 conv_time,7x7 conv_time,total_time
count,500.000000,500.000000,500.000000,500.000000
mean,60158.981323,39088.652134,91549.693108,190797.326565
std,949.448327,445.031466,1984.488615,3326.830254
min,59743.881226,38925.647736,91091.156006,189840.793610
25%,60004.234314,39011.895657,91286.778450,190348.863602
50%,60047.030449,39042.949677,91336.727142,190429.568291
75%,60095.667839,39076.209068,91393.113136,190527.319908
max,70699.453354,44404.029846,115271.329880,229441.165924


In [10]:
from util.outlier import cut_outlier